# Technical Analysis for Project 1

In [21]:
import warnings
#warnings.filterwarnings('ignore')

In [22]:
import sys
import os
import platform 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import sqlite3
import datetime

In [23]:
import yfinance as yf
import option
import utils

In [24]:
# import your own library
from stock import Stock

# switch to your own version by call TA instead of TA_AP
from TA import *
#from TA import *

In [25]:
opt = option.Option()
# set default settings

# we use relative path here, you can set it with a full path
opt.data_dir = "./data"
opt.output_dir = os.path.join(opt.data_dir, "daily")
opt.sqlite_db = os.path.join(opt.data_dir, "sqlitedb/Equity.db")

# override default option if needed, otherwise use the default settings

In [26]:
print(os.getcwd())

C:\Users\gitat\OneDrive\Documents\School\Finance_Labs\Project1


In [27]:
# check if all the attributes of the option object
print(vars(opt))

{'name': 'standard', 'user': 'gitat', 'cli_args': {}, 'environ': 'dev', 'verbose': False, 'force': False, 'start_date': None, 'end_date': None, 'tickers': None, 'port_name': None, 'data_dir': './data', 'output_dir': './data\\daily', 'sqlite_db': './data\\sqlitedb/Equity.db'}


In [28]:
ticker = 'AAPL'
db_connection = sqlite3.connect(opt.sqlite_db)
stock = Stock(opt, db_connection, ticker)

In [29]:
start_date = datetime.date(2020, 1, 1)
end_date = datetime.date(2023, 10, 1)

df = stock.get_daily_hist_price(start_date, end_date)
df.head()

,Ticker,AsOfDate,Open,High,Low,Close,Volume,Dividend,StockSplit
Date,,,,,,,,,
2020-01-02,AAPL,2020-01-02,72.246677,73.309993,71.990608,73.249023,135480400.0,0.0,0.0
2020-01-03,AAPL,2020-01-03,72.468610,73.305113,72.310090,72.536896,146322800.0,0.0,0.0
2020-01-06,AAPL,2020-01-06,71.649184,73.153913,71.395548,73.114891,118387200.0,0.0,0.0
2020-01-07,AAPL,2020-01-07,73.124639,73.383150,72.549088,72.771019,108872000.0,0.0,0.0
2020-01-08,AAPL,2020-01-08,72.471061,74.246499,72.471061,73.941650,132079200.0,0.0,0.0


***Let's do some data visualization***

In [30]:
import plotly.graph_objects as go

In [31]:
fig = go.Figure(data=go.Scatter(x=df.index,y=df['Close'], mode='lines'))
fig.show()

***Now plot the daily price as Candle stick***

https://coderzcolumn.com/tutorials/data-science/candlestick-chart-in-python-mplfinance-plotly-bokeh

In [32]:
candlestick = go.Candlestick(
                            x=df.index,
                            open=df['Open'],
                            high=df['High'],
                            low=df['Low'],
                            close=df['Close']
                            )

fig = go.Figure(data=[candlestick])

fig.show()





### Now load your TA library

In [33]:
# you will need to replace TA_AP (alex pang version) to just TA
import TA

In [34]:
# reload the module if needed
from importlib import reload
reload(TA)

<module 'TA' from 'C:\\Users\\gitat\\OneDrive\\Documents\\School\\Finance_Labs\\Project1\\TA.py'>

### Question 1: Plot the 9, 50, 100, 200 Simple Moving Averages along with the CandleSticks

In [35]:
periods = [9, 20, 50, 100, 200]
smas = SimpleMovingAverages(df, periods)
smas.run()
s1 = smas.get_series(9)
s2 = smas.get_series(50)
s3 = smas.get_series(100)
s4 = smas.get_series(200)

p9 = go.Scatter(x=s1.index,y=s1, mode='lines', line=dict(color="yellow"))
p9.name = 'SMA 9'
p50 = go.Scatter(x=s2.index,y=s2, mode='lines')
p50.name = 'SMA 50'
p100 = go.Scatter(x=s3.index,y=s3, mode='lines')
p100.name = 'SMA 100'
p200 = go.Scatter(x=s4.index,y=s4, mode='lines')
p200.name = 'SMA 200'

fig = go.Figure(data=[candlestick, p9, p50, p100, p200])
fig.show()

In [36]:
print(s1.tail())
print(s2.tail())
print(s3.tail())
print(s4.tail())

Date
2023-09-25    175.810001
2023-09-26    175.560001
2023-09-27    174.970000
2023-09-28    174.490000
2023-09-29    173.738890
Name: Close, dtype: float64
Date
2023-09-25    183.001623
2023-09-26    182.566255
2023-09-27    182.105480
2023-09-28    181.622542
2023-09-29    181.189352
Name: Close, dtype: float64
Date
2023-09-25    181.797210
2023-09-26    181.846879
2023-09-27    181.897801
2023-09-28    181.873736
2023-09-29    181.855569
Name: Close, dtype: float64
Date
2023-09-25    165.308278
2023-09-26    165.466372
2023-09-27    165.608302
2023-09-28    165.753971
2023-09-29    165.890641
Name: Close, dtype: float64


***Now plot the SMA and EMA on the same graph***

### Question 2: Plot the 50, 100 Exponential Moving Averages along with the Simple Moving Averages

In [37]:
emas = ExponentialMovingAverages(df, periods)
emas.run()
e2 = emas.get_series(50)
e3 = smas.get_series(100)

ep50 = go.Scatter(x=e2.index,y=e2, mode='lines')
ep50.name = 'EMA 50'
ep100 = go.Scatter(x=e3.index,y=e3, mode='lines')
ep100.name = 'EMA 100'

fig = go.Figure(data=[p9, p50, p100, p200, ep50, ep100])
fig.show()

### Question 3: Plot RSI with additional 70 and 30 horizontal lines

In [38]:
rsii = RSI(df)
rsii.run()
rsi = rsii.rsi

rsi_p = go.Scatter(x=rsi.index,y=rsi, mode='lines')
rsi_p.name = "RSI"

fig = go.Figure(data=[rsi_p])
fig.add_hline(y=70)
fig.add_hline(y=30)
fig.show()

### Question 4: Plot the VWAP along with the 20, 50 Simple Moving Averages and the Candlesticks

In [39]:
vwap_indicator = VWAP(df)
vwap_indicator.run()
vwap = vwap_indicator.get_series()

vwap_p = go.Scatter(x=vwap.index,y=vwap, mode='lines')
vwap_p.name = "VWAP"

s5 = smas.get_series(20)
p20 = go.Scatter(x=s5.index,y=s5, mode='lines', line=dict(color="yellow"))
p20.name = 'SMA 20'

fig = go.Figure(data=[candlestick, p20, p50, vwap_p])
fig.show()